In [ ]:
"""
SANITIZED VERSION (GitHub-safe)

What this script DOES:
- Reads private Excel files from data_private/ (not included in repo)
- Builds a long-form dataset across geometry cases
- Trains RF models for Nuavg and ΔP
- Saves models and plots to PRIVATE folders by default
- Exports parity-plot data (optional; kept private by default)

What this script DOES NOT do:
- It does not include any raw data or trained model artifacts in the public repo.
- It does not print your exact metrics by default (to keep notebook outputs clean).

Recommended repo structure:
- data_private/          (NOT in GitHub)
- models_private/        (NOT in GitHub)
- outputs_private/       (NOT in GitHub)
- src/                  (GitHub)
"""

import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt
import joblib

# -----------------------------
# 0) PATHS (PRIVATE)
# -----------------------------
DATA_DIR = "data_private"
MODELS_DIR = "models_private"
OUT_DIR = "outputs_private"

NU_FILE = os.path.join(DATA_DIR, "Nuavg.xlsx")
DP_FILE = os.path.join(DATA_DIR, "DelP.xlsx")

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

# Toggle prints to keep notebook "clean" for GitHub
VERBOSE = False

# -----------------------------
# 1) READ EXCEL (NO HEADERS)
#    Data starts at Row 1, so header=None.
# -----------------------------
nu_raw = pd.read_excel(NU_FILE, header=None, engine="openpyxl")
dp_raw = pd.read_excel(DP_FILE, header=None, engine="openpyxl")

# Enforce exactly 768 rows (do not silently drop; slice only)
nu_raw = nu_raw.iloc[:768, :].copy()
dp_raw = dp_raw.iloc[:768, :].copy()

if VERBOSE:
    print("Nuavg raw shape:", nu_raw.shape)
    print("DelP raw shape :", dp_raw.shape)

# -----------------------------
# 2) INPUT FEATURES (Columns 1–5)
#    Since header=None:
#      Column 1 -> index 0, ..., Column 5 -> index 4
# -----------------------------
X_base = nu_raw.iloc[:, 0:5].copy()
X_base.columns = ["Re", "Pr", "Da", "epsi", "Hp_mm"]

for c in X_base.columns:
    X_base[c] = pd.to_numeric(X_base[c], errors="coerce")

# -----------------------------
# 3) GEOMETRY-DEPENDENT OUTPUT COLUMN MAPS (1-based -> 0-based)
#
# Nuavg.xlsx outputs:
#   Col 5  -> a=0,   Lw N/A
#   Col 6  -> a=0.1, Lw=4
#   Col 7  -> a=0.1, Lw=5
#   Col 8  -> a=0.2, Lw=4
#   Col 9  -> a=0.2, Lw=5
#   Col 10 -> a=0.3, Lw=4
#
# DelP.xlsx outputs:
#   Col 8  -> a=0,   Lw N/A
#   Col 11 -> a=0.1, Lw=4
#   Col 14 -> a=0.1, Lw=5
#   Col 17 -> a=0.2, Lw=4
#   Col 20 -> a=0.2, Lw=5
#   Col 23 -> a=0.3, Lw=4
# -----------------------------
cases = [
    # (nu_col0, dp_col0, a_mm, Lw_mm)
    (5 - 1,  8 - 1,  0.0, 0.0),  # a=0, Lw N/A
    (6 - 1, 11 - 1,  0.1, 4.0),
    (7 - 1, 14 - 1,  0.1, 5.0),
    (8 - 1, 17 - 1,  0.2, 4.0),
    (9 - 1, 20 - 1,  0.2, 5.0),
    (10 - 1, 23 - 1, 0.3, 4.0),
]

# -----------------------------
# 4) BUILD LONG-FORM DATASET
#    Each original row becomes multiple rows (one per geometry case).
# -----------------------------
rows = []
for nu_c, dp_c, a_mm, Lw_mm in cases:
    tmp = X_base.copy()
    tmp["a_mm"] = a_mm
    tmp["Lw_mm"] = Lw_mm

    tmp["Nuavg"] = pd.to_numeric(nu_raw.iloc[:, nu_c], errors="coerce")
    tmp["DelP_Pa"] = pd.to_numeric(dp_raw.iloc[:, dp_c], errors="coerce")

    rows.append(tmp)

df = pd.concat(rows, ignore_index=True)

# -----------------------------
# 5) DROP INCOMPLETE ROWS ONLY (missing outputs or missing inputs)
# -----------------------------
df_clean = df.dropna(
    subset=["Re", "Pr", "Da", "epsi", "Hp_mm", "a_mm", "Lw_mm", "Nuavg", "DelP_Pa"]
).reset_index(drop=True)

if VERBOSE:
    print("Combined long dataset:", df.shape)
    print("Clean dataset:", df_clean.shape)
    print("Missing Nuavg:", df["Nuavg"].isna().sum())
    print("Missing DelP :", df["DelP_Pa"].isna().sum())

# Save cleaned dataset to PRIVATE output folder
clean_path = os.path.join(OUT_DIR, "ML_dataset_clean_long.xlsx")
df_clean.to_excel(clean_path, index=False)
if VERBOSE:
    print("Saved:", clean_path)

# -----------------------------
# 6) TRAIN / TEST SPLIT
# -----------------------------
features = ["Re", "Pr", "Da", "epsi", "Hp_mm", "a_mm", "Lw_mm"]
X = df_clean[features].copy()

y_nu = df_clean["Nuavg"].copy()
y_dp = df_clean["DelP_Pa"].copy()

X_train, X_test, y_nu_train, y_nu_test, y_dp_train, y_dp_test = train_test_split(
    X, y_nu, y_dp, test_size=0.2, random_state=42
)

# -----------------------------
# 7) TRAIN MODELS
# -----------------------------
rf_nu = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
)

rf_dp = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1,
)

rf_nu.fit(X_train, y_nu_train)
rf_dp.fit(X_train, y_dp_train)

# Predictions
y_nu_pred = rf_nu.predict(X_test)
y_dp_pred = rf_dp.predict(X_test)

def compute_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    return r2, mae, rmse

r2_nu, mae_nu, rmse_nu = compute_metrics(y_nu_test, y_nu_pred)
r2_dp, mae_dp, rmse_dp = compute_metrics(y_dp_test, y_dp_pred)

# Keep this OFF by default for a sanitized notebook; turn VERBOSE=True if you want.
if VERBOSE:
    print("\nMODEL PERFORMANCE (test set)")
    print(f"Nuavg: R2={r2_nu:.4f}, MAE={mae_nu:.4f}, RMSE={rmse_nu:.4f}")
    print(f"DelP (Pa): R2={r2_dp:.4f}, MAE={mae_dp:.4f}, RMSE={rmse_dp:.4f}")

# Save models to PRIVATE models folder
nu_model_path = os.path.join(MODELS_DIR, "RF_Nuavg_model.pkl")
dp_model_path = os.path.join(MODELS_DIR, "RF_DelP_model.pkl")

joblib.dump(rf_nu, nu_model_path)
joblib.dump(rf_dp, dp_model_path)

if VERBOSE:
    print("\nSaved:", nu_model_path)
    print("Saved:", dp_model_path)

# -----------------------------
# 8) PARITY PLOTS (save to PRIVATE outputs folder)
# -----------------------------
def parity_plot(y_true, y_pred, title, filename):
    plt.figure()
    plt.scatter(y_true, y_pred, s=12)
    y_min = float(min(np.min(y_true), np.min(y_pred)))
    y_max = float(max(np.max(y_true), np.max(y_pred)))
    plt.plot([y_min, y_max], [y_min, y_max])
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.show()

parity_nu_path = os.path.join(OUT_DIR, "parity_Nuavg.png")
parity_dp_path = os.path.join(OUT_DIR, "parity_DelP.png")

parity_plot(y_nu_test.values, y_nu_pred, "Parity Plot: Nuavg", parity_nu_path)
parity_plot(y_dp_test.values, y_dp_pred, "Parity Plot: ΔP (Pa)", parity_dp_path)

# -----------------------------
# 9) EXPORT TEST PREDICTIONS (PRIVATE; for MATLAB / paper plotting)
# -----------------------------
parity_df = X_test.copy()
parity_df["Nuavg_true"] = y_nu_test.values
parity_df["Nuavg_pred"] = y_nu_pred
parity_df["DelP_true"] = y_dp_test.values
parity_df["DelP_pred"] = y_dp_pred

parity_xlsx_path = os.path.join(OUT_DIR, "test_predictions_parity_data.xlsx")
parity_df.to_excel(parity_xlsx_path, index=False)

if VERBOSE:
    print("Saved:", parity_xlsx_path)

# -----------------------------
# 10) OPTIONAL: SAVE METRICS SUMMARY (PRIVATE)
# -----------------------------
metrics_summary = pd.DataFrame(
    {
        "target": ["Nuavg", "DelP_Pa"],
        "R2": [r2_nu, r2_dp],
        "MAE": [mae_nu, mae_dp],
        "RMSE": [rmse_nu, rmse_dp],
        "n_train": [len(X_train), len(X_train)],
        "n_test": [len(X_test), len(X_test)],
    }
)
metrics_path = os.path.join(OUT_DIR, "metrics_summary.xlsx")
metrics_summary.to_excel(metrics_path, index=False)

if VERBOSE:
    print("Saved:", metrics_path)

# If you want a minimal "sanitized" confirmation message without numbers:
print("Done. (Models/plots exported to private folders; data not included for GitHub.)")
